In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

In [6]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [36]:
%%cu
#include<stdio.h>
#include<stdlib.h>
__global__ void gpu_add(int* A,int* B, int* C , int M){
    int thread = threadIdx.x + blockIdx.x *blockDim.x;
    if(thread < M){
    C[thread] = A[thread] + B[thread];
    }
}
int main(){
    int *h_a , *h_b , *h_c;
    int m = 10;
    /* printf("Enter the size of 1D array required for random number initialization\n");
    scanf("%d",&m); */
    cudaMallocHost((void **)&h_a , sizeof(int)*m);
    cudaMallocHost((void **)&h_b , sizeof(int)*m);
    cudaMallocHost((void **)&h_c , sizeof(int)*m);

    for (int i =0 ; i<m ; i++){
        h_a[i] = rand() %100;
        h_b[i] = rand() %100;
    }
    printf("Input String 1: \n");
    for (int j =0; j < m ; j++){
        printf("%d ",h_a[j]);
    }

    printf("\nInput String 2: \n");
        for (int j =0; j < m ; j++){
        printf("%d ",h_b[j]);
    }
    int *a,*b,*c;
    cudaMalloc((void **)&a,sizeof(int)*m);
    cudaMalloc((void **)&b,sizeof(int)*m);
    cudaMalloc((void **)&c,sizeof(int)*m);
    cudaMemcpy(a,h_a,sizeof(int)*m,cudaMemcpyHostToDevice);
    cudaMemcpy(b,h_b,sizeof(int)*m,cudaMemcpyHostToDevice);
    gpu_add<<<ceil(m/256.0),256>>>(a,b,c,m);

    cudaMemcpy(h_c,c,sizeof(int)*m,cudaMemcpyDeviceToHost);
    printf("\nResult is: \n");
    for(int i=0;i<m;i++){
        printf("%d ",h_c[i]);
    }
}

Input String 1: 
83 77 93 86 49 62 90 63 40 72 
Input String 2: 
86 15 35 92 21 27 59 26 26 36 
Result is: 
169 92 128 178 70 89 149 89 66 108 
